In [2]:
import torch
import torch.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
from skimage.transform import resize
import argparse
from PIL import Image

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def caption_image_beam_search(encoder, decoder, image_path, word_map, beam_size = 3):
    """
    Reads an image and captions it with beam search
    
    :param encoder: encoder model 
    :param decoder: decoder model
    :param image_path: path to image
    :param word_map: word map
    :param beam_size: number of sequences to consider at each decode-step
    
    :returns: caption, weights for visualization
    """
    
    k = beam_size
    vocab_size = len(word_map)
    
    img = plt.imread(image_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis()]
        img = np.concatenate([img, img, img], axis = 2)
    
    img = resize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    transform = transforms.Compose([normalize])
    image = transform(img)  
    
    #Encode:
    image = image.unsqueeze(0) #(1, 3, 256, 256)
    encoder_out = encoder(image) #(1, enc_image_size, enc_image_size, enc_dim)
    enc_image_size = encoder_out.size(1)
    enc_dim = encoder_out.size(3)
    
    #Flatten Encoding:
    encoder_out = encoder_out.view(1, -1, enc_dim) #(1, num_pixels, enc_dim)
    num_pixels = encoder_out.view(1)
    
    #We'll treat the problem as if we'rehaving a batch of size k:
    encoder_out = encoder_out.expand(k, num_pixels, enc_dim)
    
    #Tensor to store top k previous words. Now they're just start:
    k_prev_words = torch.LongTensor([[word_map["<start>"]]] * k).to(device) #(k, 1)
    
    #Tensor to store top k sequences:
    seqs = k_prev_words #(k, 1)
    
    #Tensor to store top k sequences scores. Now they're just 0:
    top_k_scores = torch.zeros(k, 1).to(device)
    
    #Tensor to store top k sequences alphas. Now they're just 1:
    seqs_alpha = torch.ones(k, 1, enc_image_size, enc_image_size).to(device) #(k, 1, enc_image_size, enc_image_size)
    
    #Lists to store complete sequences, their alphas and scores
    complete_seqs = list()
    complete_seqs_alpha = list()
    complete_seqs_scores = list()
    
    #Start decoding:
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)
    
    # s is a number less than or equal to k, 
    # because sequences are removed from this process once they hit <end>
    # TODO: I think this is the batch_size. That would make sense if you take a look at models.py 
    while True:
        embeddings = decoder.embedding(k_prev_words).squeeze(1) #(s, embed_dim)
        awe, alpha = decoder.attention(encoder_out, h) #(s, encoder_dim), (s, num_pixels)
        alpha = alpha.view(-1, enc_image_size, enc_image_size) #(s, enc_image_size, enc_image_size)
        gate = decoder.sigmoid(decoder.f_beta(h)) #(s, encoder_dim)
        awe = gate * awe
        
        h, c = decoder.decode_step(torch.cat([embeddings, awe], dim = 1), (h, c))
        
        scores = decoder.fc(h) #(s, vocab_size)
        scores = F.log_softmax(scores, dim=1)
        
        #Add scores:
        scores = top_k_scores.expand_as(scores) + scores # (s, vocab_size)
        
        # For the first step, all k points will have the same scores (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
        else:
            # Unroll and find top scores, and their unrolled indices
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)
            
        # Convert unrolled indices to actual indices of scores
        #TODO: Why is this done like this?
        prev_word_inds = top_k_words / vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)
        
        #Add new words to sequences, alphas. Step represents length of sentence generated so far.
        # second dim is (step + 1), because of the <start> token.
        seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1) #(s, step+1)
        seqs_alpha = torch.cat([seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)],
                               dim=1)  # (s, step+1, enc_image_size, enc_image_size)
        
        #Which sequences are incomplete, didn't reach <end>?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if next_word != word_map['<end>']]
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))
        
        # Set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist())
            complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        if k == 0:
            break
        
        seqs = seqs[incomplete_inds]
        seqs_alpha = seqs_alpha[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)
        
        if step > 50:
            break
        
        step += 1
    
    #Collecting indexes with max scores
    i = complete_seqs_scores.index(max(complete_seqs_scores))
    #Collecting sequences of those indexes
    seq = complete_seqs[i]
    #Collecting weights of those indexes
    alphas = complete_seqs_alpha[i]

    return seq, alphas


def visualize_att(image_path, seq, alphas, rev_word_map, smooth=True):
    """
    Visualizes caption with weights at every word.
    Adapted from paper authors' repo: https://github.com/kelvinxu/arctic-captions/blob/master/alpha_visualization.ipynb
    :param image_path: path to image that has been captioned
    :param seq: caption
    :param alphas: weights
    :param rev_word_map: reverse word mapping, i.e. ix2word
    :param smooth: smooth weights?
    """
    image = Image.open(image_path)
    image = image.resize([14 * 24, 14 * 24], Image.LANCZOS)

    words = [rev_word_map[ind] for ind in seq]

    for t in range(len(words)):
        if t > 50:
            break
        plt.subplot(np.ceil(len(words) / 5.), 5, t + 1)

        plt.text(0, 1, '%s' % (words[t]), color='black', backgroundcolor='white', fontsize=12)
        plt.imshow(image)
        current_alpha = alphas[t, :]
        if smooth:
            alpha = skimage.transform.pyramid_expand(current_alpha.numpy(), upscale=24, sigma=8)
        else:
            alpha = skimage.transform.resize(current_alpha.numpy(), [14 * 24, 14 * 24])
        if t == 0:
            plt.imshow(alpha, alpha=0)
        else:
            plt.imshow(alpha, alpha=0.8)
        plt.set_cmap(cm.Greys_r)
        plt.axis('off')
    plt.show()


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Show, Attend, and Tell - Tutorial - Generate Caption')

    parser.add_argument('--img', '-i', help='path to image')
    parser.add_argument('--model', '-m', help='path to model')
    parser.add_argument('--word_map', '-wm', help='path to word map JSON')
    parser.add_argument('--beam_size', '-b', default=5, type=int, help='beam size for beam search')
    parser.add_argument('--dont_smooth', dest='smooth', action='store_false', help='do not smooth alpha overlay')

    args = parser.parse_args()

    # Load model
    checkpoint = torch.load(args.model, map_location=str(device))
    decoder = checkpoint['decoder']
    decoder = decoder.to(device)
    decoder.eval()
    encoder = checkpoint['encoder']
    encoder = encoder.to(device)
    encoder.eval()

    # Load word map (word2ix)
    with open(args.word_map, 'r') as j:
        word_map = json.load(j)
    rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

    # Encode, decode with attention and beam search
    seq, alphas = caption_image_beam_search(encoder, decoder, args.img, word_map, args.beam_size)
    alphas = torch.FloatTensor(alphas)

    # Visualize caption and attention of best sequence
    visualize_att(args.img, seq, alphas, rev_word_map, args.smooth)

usage: ipykernel_launcher.py [-h] [--img IMG] [--model MODEL] [--word_map WORD_MAP] [--beam_size BEAM_SIZE] [--dont_smooth]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/aizenangel/.local/share/jupyter/runtime/kernel-a345fa6e-3ac6-4f1c-b017-8db6bb8fe4f6.json


SystemExit: 2

/home/aizenangel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
